In [25]:
import pandas as pd
import numpy as np
import csv
from pandasql import sqldf

In [26]:
customer_list = pd.read_table('user_list_1.txt',sep='\t')  
print(customer_list)

                                                  org_id  remap_id
0      be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee9...         0
1      b4db5e5259234574edfff958e170fe3a5e13b6f146752c...         1
2      49beaacac0c7801c2ce2d189efe525fe80b5d37e46ed05...         2
3      a65f77281a528bf5c1e9f270141d601d116e1df33bf9df...         3
4      cd04ec2726dd58a8c753e0d6423e57716fd9ebcf2f14ed...         4
...                                                  ...       ...
24646  ed93522b580ceaf459a259e40d351f2edd0a99d400ba1f...     24646
24647  ef63632da96646e195a1f79b08514262f39684db0d9510...     24647
24648  dffe5c5f19bd5a3c6de4fc4f23d2167e3f036544d374fa...     24648
24649  91d2b7645e9e9bc088bba9436c4b8f1d48f640b6e024df...     24649
24650  f728f2ccaa755663806a775574d74973202b9c3ad19555...     24650

[24651 rows x 2 columns]


In [27]:
transaction = pd.read_csv("transactions_train.csv", header=0)
last_two_weeks_tran = transaction[transaction["t_dat"].str.match(pat = '((2020-09-1)|(2020-09-2).)')]
last_two_weeks_tran

,t_dat,customer_id,article_id,price,sales_channel_id
31334398,2020-09-10,000fb6e772c5d0023892065e659963da90b1866035558e...,884319006,0.025407,2
31334399,2020-09-10,000fb6e772c5d0023892065e659963da90b1866035558e...,762846026,0.025407,2
31334400,2020-09-10,000fb6e772c5d0023892065e659963da90b1866035558e...,919273004,0.042356,2
31334401,2020-09-10,000fb6e772c5d0023892065e659963da90b1866035558e...,919273004,0.042356,2
31334402,2020-09-10,000fb6e772c5d0023892065e659963da90b1866035558e...,927922002,0.033881,2
...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1


In [28]:
new_transaction = pd.DataFrame(last_two_weeks_tran, columns=["customer_id", "article_id"])
new_transaction

,customer_id,article_id
31334398,000fb6e772c5d0023892065e659963da90b1866035558e...,884319006
31334399,000fb6e772c5d0023892065e659963da90b1866035558e...,762846026
31334400,000fb6e772c5d0023892065e659963da90b1866035558e...,919273004
31334401,000fb6e772c5d0023892065e659963da90b1866035558e...,919273004
31334402,000fb6e772c5d0023892065e659963da90b1866035558e...,927922002
...,...,...
31788319,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001
31788320,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004
31788321,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001
31788322,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002


In [29]:
customer_purchase_detail = sqldf("""SELECT remap_id as customer, article_id 
        FROM customer_list as c
        INNER JOIN new_transaction as t
        on c.org_id = t.customer_id
        group by remap_id, article_id
    """)
print(customer_purchase_detail)

        customer  article_id
0              0   730683054
1              0   748566027
2              0   816746001
3              0   863646001
4              0   864224005
...          ...         ...
198132     24650   791486009
198133     24650   802965009
198134     24650   835522002
198135     24650   875117001
198136     24650   902443002

[198137 rows x 2 columns]


In [30]:
item_list = pd.read_table('item_list_1.txt',sep='\t')  
# item_list
customer_purchase_remapid = sqldf("""SELECT customer, remap_id 
        FROM customer_purchase_detail as c
        INNER JOIN item_list as i
        on c.article_id = i.org_id
    """)
print(customer_purchase_remapid)

        customer  remap_id
0              0       713
1              0      2067
2              0      2553
3              0        26
4              0      1442
...          ...       ...
198132     24650     16092
198133     24650      8020
198134     24650     19433
198135     24650     18526
198136     24650     20301

[198137 rows x 2 columns]


In [31]:
f = open('total.txt','a')
for i in range(24651): # 24651 is the total number of the customers
    cur_line = customer_purchase_remapid[customer_purchase_remapid['customer'] == i]
    if cur_line.empty:
        continue
    cur_line_new = np.asarray(cur_line['remap_id'])
    str_cur_line = " ".join([str(x) for x in cur_line_new])
    str_cur_line = str(i) + " " + str_cur_line
    f.write(str_cur_line)
    f.write("\n")
 
f.close()

In [32]:
f1 = open('train_1.txt','a')
f2 = open('test_1.txt', 'a')
for i in range(24651):
    cur_line = customer_purchase_remapid[customer_purchase_remapid['customer'] == i]
    if cur_line.empty:
        continue
    cur_line_new = np.asarray(cur_line['remap_id'])
    if len(cur_line_new) < 5:
        continue
    else:
        train_part_index = np.random.choice(cur_line_new.shape[0], size=int(0.8 * len(cur_line_new)), replace=False)
        train_part = cur_line_new[train_part_index]
        str_cur_train = " ".join([str(x) for x in train_part])
        str_cur_train = str(i) + " " + str_cur_train
        f1.write(str_cur_train)
        f1.write("\n")
        test_part_index = np.delete(np.arange(cur_line_new.shape[0]), train_part_index)
        test_part = cur_line_new[test_part_index]
        str_cur_test = " ".join([str(x) for x in test_part])
        str_cur_test = str(i) + " " + str_cur_test
        f2.write(str_cur_test)
        f2.write("\n")
        
f1.close()
f2.close()